# Question Classification

### Data Preparation and Cleaning

In [1]:
#read data
texts = []
labels = []

with open('data/LabelledData.txt','r') as f:
    for line in f:
        text, label = map(str,line.split(",,,"))
        texts.append(text.strip())
        labels.append(label.strip())

In [2]:
#for text, label in zip(texts[:10], labels[:10]):
#    print(text," -->", label)

In [3]:
#clean the sentences
import re
def pre_process(text):
    text = re.sub(r"\b's\b","is",text)
    text = re.sub(r"[^a-z?\.]"," ",text.lower())
    return text

In [4]:
processed_texts = [pre_process(text) for text in texts]

In [5]:
for text, label in zip(processed_texts[:10], labels[:10]):
    print(text," -->", label)

how did serfdom develop in and then leave russia ?  --> unknown
what films featured the character popeye doyle ?  --> what
how can i find a list of celebrities   real names ?  --> unknown
what fowl grabs the spotlight after the chinese year of the monkey ?  --> what
what is the full form of .com ?  --> what
what contemptible scoundrel stole the cork from my lunch ?  --> what
what team did baseball  s st. louis browns become ?  --> what
what is the oldest profession ?  --> what
what are liver enzymes ?  --> what
name the scar faced bounty hunter of the old west .  --> unknown


In [6]:
import numpy as np

X = np.array(texts)
y = np.array(labels, dtype='str')

In [7]:
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

{'affirmation': 104, 'unknown': 272, 'what': 609, 'when': 96, 'who': 402}

### Data Preprocessing

In [8]:
#import some keras libraries
from sklearn.preprocessing import LabelBinarizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import os, sys

from keras.layers import Dense, Input, GlobalMaxPooling1D, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model, Sequential
from keras import utils
from keras.layers import concatenate, Activation
from keras.callbacks import ModelCheckpoint

/home/himanshu/anaconda3/envs/deeplearning/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Format our text samples and labels into tensors that can be fed into a neural network

In [9]:
MAX_SEQUENCE_LENGTH = 45
MAX_NUM_WORDS = 1000
TESTING_SPLIT = 0.1

In [10]:
# Split data into train and test
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
data = X[indices]
labels = y[indices]
num_testing_samples = int(TESTING_SPLIT * X.shape[0])

In [11]:
train_x = data[:-num_testing_samples]
train_y = labels[:-num_testing_samples]
test_x = data[-num_testing_samples:]
test_y = labels[-num_testing_samples:]

In [12]:
#Binarize labels in a one-vs-all fashion
encoder = LabelBinarizer()
encoder.fit(train_y)
y_train = encoder.transform(train_y)
y_test = encoder.transform(test_y)

In [13]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_x)
x_train = tokenizer.texts_to_matrix(train_x)
x_test = tokenizer.texts_to_matrix(test_x)

In [14]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (1335, 1000)
x_test shape: (148, 1000)
y_train shape: (1335, 5)
y_test shape: (148, 5)


In [16]:
batch_size = 32
epochs = 5

In [17]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 3427 unique tokens.


### Training

No need of embedding layer(mapping semantic meaning into a geometric space).

To define the layers of our model we will use the Keras Sequential model. We will use the cross entropy loss function, since each of our question can only belong to one tag. 

In [18]:
model = Sequential()
model.add(Dense(512, input_shape=(MAX_NUM_WORDS,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               512512    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 1285      
__________

In [19]:
#save the weights
filepath="weights/best_weights.{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, 
                             save_best_only=True, mode='max')

In [20]:
#fit the model on training set with validation split 0.1. Don't touch testing set
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 1201 samples, validate on 134 samples
Epoch 1/5
1201/1201 [==============================] - 1s 883us/step - loss: 1.0139 - acc: 0.6436 - val_loss: 0.4888 - val_acc: 0.8433
Epoch 2/5
1201/1201 [==============================] - 1s 695us/step - loss: 0.2716 - acc: 0.9334 - val_loss: 0.2193 - val_acc: 0.9478
Epoch 3/5
1201/1201 [==============================] - 1s 649us/step - loss: 0.1014 - acc: 0.9709 - val_loss: 0.1598 - val_acc: 0.9552
Epoch 4/5
1201/1201 [==============================] - 1s 742us/step - loss: 0.0514 - acc: 0.9875 - val_loss: 0.1604 - val_acc: 0.9552
Epoch 5/5
1201/1201 [==============================] - 1s 666us/step - loss: 0.0259 - acc: 0.9933 - val_loss: 0.1446 - val_acc: 0.9776


After 5 epochs:
- loss: 0.075
- accuracy: 0.9850
- val_loss: 0.1217

Evalaute on testing dataset 

In [21]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

148/148 [==============================] - 0s 171us/step
Test score: 0.12457356752978789
Test accuracy: 0.9662162162162162


### Generating the predictions

In [22]:
text_labels = encoder.classes_

In [23]:
#print the test cases where it fails
for i in range(148):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    if test_y[i]!=predicted_label:
        print(test_x[i][:100], "...")
        print('Actual label:' + test_y[i])
        print("Predicted label: " + predicted_label + "\n")

what does a nihilist believe in ? ...
Actual label:unknown
Predicted label: what

name the ranger who was always after yogi bear . ? ...
Actual label:unknown
Predicted label: who

what actor came to dinner in guess who 's coming to dinner ? ...
Actual label:who
Predicted label: what

do i need to tear off the black outside before i put the new filter into the machine ? ...
Actual label:affirmation
Predicted label: unknown

can i install on wall ? ...
Actual label:affirmation
Predicted label: unknown

